In [ ]:
'''
Goal: Run SnapATAC2 pipeline on multiome snATAC data
Author:Carsten Knutsen
Date:231016
conda_env:snapatac
'''

In [ ]:
import scanpy as sc
import muon as mu
import numpy as np
import pandas as pd
import snapatac2 as snap
import os

In [ ]:
output_fol = '/home/carsten/alvira_bioinformatics/postnatal_lung_multiome/data/pilot/231016_snapatac2_pilot'
os.makedirs(output_fol, exist_ok=True)

In [ ]:
adata_rna = sc.read('/home/carsten/alvira_bioinformatics/postnatal_lung_multiome/data/single_cell_files/share/p7_multiome_rna_processed.gz.h5ad')
adata_rna.obs_names

In [ ]:
fragment_file = '/home/carsten/alvira_bioinformatics/postnatal_lung_multiome/data/cellranger_output/230609_aggregate/outs/atac_fragments.tsv.gz'
output_f = f'{output_fol}/multiome_snapatac_pilot.h5ad'
gtf = '/home/carsten/alvira_bioinformatics/postnatal_lung_multiome/data/refdata-cellranger-arc-mm10-2020-A-2.0.0/genes/genes.gtf'
chrom_sizes_fn = '/home/carsten/alvira_bioinformatics/postnatal_lung_multiome/data/refdata-cellranger-arc-mm10-2020-A-2.0.0/sizes.genome'
genome = '/home/carsten/alvira_bioinformatics/postnatal_lung_multiome/data/refdata-cellranger-arc-mm10-2020-A-2.0.0/fasta/genome.fa'

In [ ]:
## Chrom sizes were genereated using command on next line 
# faidx refdata-cellranger-arc-mm10-2020-A-2.0.0/fasta/genome.fa -i chromsizes > sizes.genome
chrom_sizes = pd.read_csv(chrom_sizes_fn, sep='\t', header=None, index_col=0).to_dict()
chrom_sizes = chrom_sizes[1]

In [ ]:
%%time
data = snap.pp.import_data(
    fragment_file,
    chrom_sizes,
    file=output_f,  # Optional
    sorted_by_barcode=False,
)
data

In [ ]:
fig = snap.pl.frag_size_distr(data, show=False)
fig.update_yaxes(type="log")
fig.show()

In [ ]:
%%time
snap.metrics.tsse(data, gtf)

In [ ]:
snap.pl.tsse(data, interactive=False)

In [ ]:
%%time
snap.pp.filter_cells(data, min_counts=1000, min_tsse=0, max_counts=100000)
overlap_names = list(set(adata_rna.obs_names) & set(data.obs_names))
data.subset(obs_indices=overlap_names)
data.obs['lineage'] = adata_rna.obs['lineage'].loc[overlap_names]
data.obs['celltype'] = adata_rna.obs['celltype'].loc[overlap_names]
data.obs['treatment'] = adata_rna.obs['treatment'].loc[overlap_names]
data.obs['mouse'] = adata_rna.obs['mouse'].loc[overlap_names]
data.obs['sex'] = adata_rna.obs['sex'].loc[overlap_names]

In [ ]:
%%time
snap.pp.add_tile_matrix(data)

In [ ]:
%%time
snap.pp.select_features(data, n_features=500000)
snap.tl.spectral(data)
snap.pp.knn(data)
snap.tl.leiden(data)
snap.tl.umap(data)
snap.pl.umap(data, color='leiden', interactive=False, height=500)




In [ ]:
%%time
gene_matrix = snap.pp.make_gene_matrix(data, gtf)
gene_matrix

In [ ]:
import scanpy as sc

sc.pp.filter_genes(gene_matrix, min_cells= 5)
sc.pp.normalize_total(gene_matrix)
sc.pp.log1p(gene_matrix)

In [ ]:
%load_ext autoreload


In [ ]:
%autoreload 2

In [ ]:
%%time
import scanpy.external as sce
sce.pp.magic(gene_matrix, solver="approximate")

In [ ]:
snap.tl.macs3(data, groupby='celltype')


In [ ]:
%%time
peaks = snap.tl.merge_peaks(data.uns['macs3'], chrom_sizes)
peaks.head()

In [ ]:
%%time
peak_mat = snap.pp.make_peak_matrix(data, use_rep=peaks['Peaks'])
peak_mat

In [ ]:
%%time
marker_peaks = snap.tl.marker_regions(peak_mat, groupby='celltype', pvalue=0.01)

In [ ]:
marker_peaks

In [ ]:
%%time
motifs = snap.tl.motif_enrichment(
    motifs=snap.datasets.cis_bp(unique=True),
    regions=marker_peaks,
    genome_fasta=genome,
)

In [ ]:
snap.pl.regions(peak_mat, groupby='celltype', peaks=marker_peaks, interactive=False)


In [ ]:
snap.pl.motif_enrichment(motifs, max_fdr=0.0001, height=1600, interactive=False)


In [ ]:
motifs

In [ ]:
# Copy over UMAP embedding
gene_matrix.obsm["X_umap"] = data.obsm["X_umap"]

In [ ]:
snap.pp.select_features(data, n_features=250000)
snap

In [ ]:
%%time
snap.pp.scrublet(data)

In [ ]:
snap.pp.filter_doublets(data)
data

In [ ]:
data

In [ ]:
data.subset(obs_indices=list(set(adata_rna.obs_names) & set(data.obs_names)))
data.obs['lineage'] = adata_rna.obs['lineage'].loc[overlap_names]
data.obs['celltype'] = adata_rna.obs['celltype'].loc[overlap_names]
data.obs['treatment'] = adata_rna.obs['treatment'].loc[overlap_names]
data.obs['mouse'] = adata_rna.obs['mouse'].loc[overlap_names]
data.obs['sex'] = adata_rna.obs['sex'].loc[overlap_names]

In [ ]:
%%time
snap.pp.add_tile_matrix(data)

In [ ]:
snap.pp.select_features(data, n_features=250000)


In [ ]:
snap.pp.harmony(data, batch='mouse')

In [ ]:
data

In [ ]:
%%time
snap.tl.umap(data, use_rep='X_spectral_harmony',)

In [ ]:
snap.pp.knn(data,use_rep ='X_spectral_harmony' )
snap.tl.leiden(data)

In [ ]:
snap.pl.umap(data, color='leiden', interactive=False, height=500)


In [ ]:
%%time
marker_peaks = snap.tl.marker_regions(peak_mat, groupby='celltype', pvalue=0.01)

In [ ]:
snap.pl.umap(data, color='lineage', interactive=False, height=500)


In [ ]:
data.close()

In [ ]:
adata = sc.read(output_f)

In [ ]:
data = snap.read(output_f)
snap.pl.umap(data, color='celltype', interactive=False, height=500)


In [ ]:
snap.pl.umap(data, color='mouse', interactive=False, height=500)


In [ ]:
%%time
snap.tl.call_peaks(data, groupby='celltype')

In [ ]:
%%time
peak_mat = snap.pp.make_peak_matrix(data, 
                                    file="/home/carsten/alvira_bioinformatics/postnatal_lung_multiome/data/single_cell_files/multiome_snapatac_peak_matrix.h5ad"
                                   )
peak_mat

In [ ]:
snap.pl.regions(peak_mat, groupby='celltype', peaks=marker_peaks, interactive=False)


In [ ]:
peak_mat = snap.read("/home/carsten/alvira_bioinformatics/postnatal_lung_multiome/data/single_cell_files/multiome_snapatac_peak_matrix.h5ad")
%%time
marker_peaks = snap.tl.marker_regions(peak_mat, groupby='celltype', pvalue=0.01)


In [ ]:
peak_mat

In [ ]:
%%time
motifs = snap.tl.motif_enrichment(
    motifs=snap.datasets.cis_bp(unique=True),
    regions=marker_peaks,
    genome_fasta=genome,
)

In [ ]:
motifs['Venous EC'].to_pandas()

In [ ]:
with pd.ExcelWriter(
    f"{output_fol}/motif_enrichment_celltype.xlsx", engine="xlsxwriter"
) as writer:
    for key in sorted(motifs.keys()):
        out = key.replace('/','_')
        df = motifs[key].to_pandas()
        df.loc[df['adjusted p-value']<0.01].sort_values('log2(fold change)', ascending=False).to_excel(writer, sheet_name=out[:31])

In [ ]:
diff_peaks_dt = {}
for ct in adata_rna.obs['celltype'].cat.categories:
    print(ct)
    norm = (data.obs['celltype'] == ct)&(data.obs['treatment'] == 'Normoxia')
    hyper = (data.obs['celltype'] == ct)&(data.obs['treatment'] == 'Hyperoxia')
    peaks_selected = np.logical_or(
    data.uns["peaks"][ct].to_numpy(),
    data.uns["peaks"][ct].to_numpy(),
)

    diff_peaks = snap.tl.diff_test(
    peak_mat,
    cell_group1=norm,
    cell_group2=hyper,
        features=data.uns["peaks"][ct].to_numpy()
)
    diff_peaks = diff_peaks.to_pandas()
    difF_peaks_dt[ct] = diff_peaks

In [ ]:
norm = (data.obs['celltype'] == 'AT1')&(data.obs['treatment'] == 'Normoxia')
hyper = (data.obs['celltype'] == 'AT2')&(data.obs['treatment'] == 'Normoxia')
peaks_selected = np.logical_or(
data.uns["peaks"]['AT1'].to_numpy(),
data.uns["peaks"]['AT2'].to_numpy(),
)

diff_peaks = snap.tl.diff_test(
peak_mat,
cell_group1=norm,
cell_group2=hyper,
    features=peaks_selected,
    penalty='none'
)
diff_peaks = diff_peaks.to_pandas()
difF_peaks_dt

In [ ]:
snap.pl.motif_enrichment(motifs, max_fdr=0.0001, height=3000, width = 2000,interactive=False)


In [ ]:
rna_names=adata_rna.obs_names.tolist()
overlap_names = list(set(adata_rna.obs_names) & set(data.obs_names))
not_in_fragment = [x for x in rna_names if x not in overlap_names]

In [ ]:
adata_rna.obs.loc[not_in_fragment]

In [ ]:
not_in_fragment

In [ ]:
print(len(adata_rna.obs_names.tolist()))
print(len(data.obs_names))
print(len(list(set(adata_rna.obs_names) & set(data.obs_names))))


In [ ]:
snap.pl.tsse(data, interactive=False)

In [ ]:
data

In [ ]:
len(set(data.obs_names))

In [ ]:
data_sub = 
data_sub

In [ ]:
%%time
snap.pp.filter_cells(data, min_counts=5000, min_tsse=0, max_counts=1000000)
data